In [6]:
import asyncio
import socketio
import random
import json
sio = socketio.AsyncClient()

map = []
snakes = {}
foods = []
player_id = None
FRAME_RATE = 0.1


def direction_to_string(direction):
    if direction == [0, -1]:
        return "up"
    if direction == [0, 1]:
        return "down"
    if direction == [-1, 0]:
        return "left"
    if direction == [1, 0]:
        return "right"
    return None


@sio.event
async def connect():
    print("Connected to the server")
    await sio.emit("register", {"player_id": player_id})


@sio.event
async def disconnect():
    print("Disconnected from the server")


@sio.event
async def state(data):
    global map, snakes, foods, player_id
    data = json.loads(data)
    map = data["map"]
    snakes = data["snakes_pos"]
    foods = data["foods_pos"]
    
    if player_id is None: return
    
    if str(player_id) not in snakes.keys():
        
        await sio.emit("exit")
        await sio.disconnect()
        return
    player_pos = snakes[str(player_id)]["body"][0]
    player_direction = direction_to_string(snakes[str(player_id)]["direction"])
    print(player_pos, player_direction)


@sio.event
async def register(data):
    print("Registered as player", data["player_id"])
    global player_id
    player_id = data["player_id"]
    print(f"Registered as player {player_id}")
    # After registration, start sending moves
    asyncio.create_task(send_moves())


async def send_moves():
    
    while True:
        global map, snakes, player_id, foods
        # Find the player's current position
        player_pos = None
        player_direction = None
        if str(player_id) in snakes.keys():
            player_pos = snakes[str(player_id)]["body"][0]
            player_direction = direction_to_string(snakes[str(player_id)]["direction"])
        # If the player's position was not found, don't send a move message
        if player_pos is None or player_direction is None:
            await asyncio.sleep(FRAME_RATE)
            continue

        # print (player_pos, player_direction)
        # Determine possible moves
        directions = []
        if (
            player_pos[0] > 0
            and (
                map[player_pos[0] - 1][player_pos[1]] == ""
                or map[player_pos[0] - 1][player_pos[1]] == "food"
            )
            and player_direction != "right"
        ):
            directions.append("left")
        if (
            player_pos[0] < len(map) - 1
            and (
                map[player_pos[0] + 1][player_pos[1]] == ""
                or map[player_pos[0] + 1][player_pos[1]] == "food"
            )
            and player_direction != "left"
        ):
            directions.append("right")
        if (
            player_pos[1] > 0
            and (
                map[player_pos[0]][player_pos[1] - 1] == ""
                or map[player_pos[0]][player_pos[1] - 1] == "food"
            )
            and player_direction != "down"
        ):
            directions.append("up")
        if (
            player_pos[1] < len(map[0]) - 1
            and (
                map[player_pos[0]][player_pos[1] + 1] == ""
                or map[player_pos[0]][player_pos[1] + 1] == "food"
            )
            and player_direction != "up"
        ):
            directions.append("down")

        # If there are no possible moves, don't send a move message
        if not directions:
            await asyncio.sleep(FRAME_RATE)
            continue

        # Choose a random direction from the possible moves
        random_direction = random.choice(directions)
        message = {"type": "move", "direction": random_direction}
        await sio.emit("move", message)
        await asyncio.sleep(FRAME_RATE)  # pause for a second between messages


async def main():
    global player_id
    player_id = input("Please enter your player ID: ")
    await sio.connect("https://ntuim-multiplayer-snake-game-server.azurewebsites.net/")
    # await sio.connect("http://localhost:3000")
    await sio.wait()


import nest_asyncio

nest_asyncio.apply()

await main()

Connected to the server
Registered as player yym
Registered as player yym
[15, 5] right
[16, 5] right
[16, 4] up
[15, 4] left
[15, 3] up
[14, 3] left
[14, 2] up
[14, 1] up
[15, 1] right
[16, 1] right
[17, 1] right
[17, 0] up
[18, 0] right
[18, 1] down
[18, 2] down
[18, 3] down
[18, 4] down
[18, 5] down
[17, 5] left
[16, 5] left
[15, 5] left
[15, 4] up
[14, 4] left
[14, 5] down
[14, 6] down
[14, 7] down
[15, 7] right
[16, 7] right
[16, 8] down
[15, 8] left
[15, 7] up
[16, 7] right
[17, 7] right
[17, 8] down
[17, 9] down
[16, 9] left
[16, 8] up
[16, 7] up
[16, 6] up
[16, 5] up
[17, 5] right
[18, 5] right
[18, 6] down
[19, 6] right
[19, 5] up
[19, 4] up
[19, 3] up
[19, 2] up
[18, 2] left
[17, 2] left
[17, 1] up
[16, 1] left
[16, 2] down
[17, 2] right
[17, 3] down
[17, 4] down
[18, 4] right
[18, 3] up
[18, 2] up
[18, 1] up
[18, 0] up
[19, 0] right
[19, 1] down
[19, 2] down
[19, 3] down
[19, 4] down
[19, 5] down
[19, 6] down
[18, 6] left
[17, 6] left
[16, 6] left
[16, 5] up
[17, 5] right
[1